# Setup
This IPython notebook will walk through the steps of characterizing iModulons through the semi-automated tools in PyModulon. You will need:

* M and A matrices
* Expression data (e.g. `log_tpm_norm.csv`)
* Gene table and KEGG/GO annotations (Generated in `gene_annotation.ipynb`)
* Sample table, with a column for `project` and `condition`
* TRN file

Optional:
* iModulon table (if you already have some characterized iModulons)

In [1]:
from pymodulon.core import IcaData
from pymodulon.plotting import *
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm

In [2]:
# Enter the location of your data here
work_dir = '/home/jovyan/work/'
data_dir = path.join(work_dir,'processed_data')

# GO and KEGG annotations are in the 'external' folder
external_data = path.join(work_dir,'external')

## Check your sample table (i.e. metadata file)
Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [37]:
df_metadata = pd.read_csv(path.join(data_dir,'metadata.tsv'),index_col=0,sep='\t')

for idx, row in df_metadata[df_metadata['treatment'].isnull()].iterrows():
    df_metadata.loc[idx, 'treatment'] = row.condition
df_metadata.drop('condition', axis=1, inplace=True)

In [38]:
#combine different values to create unique condition names
cat_cols = ['treatment', 'growth_phase', 'isolate', 'genetic_changes'] 

df_metadata['condition'] = df_metadata['base_media'].str.cat(df_metadata[cat_cols],sep=';')

In [43]:
df_metadata[['project','condition']].head()

,project,condition
CM_1,CcpA CodY interaction,CA-MHB;Control;exp;JE2;WT
CM_10,CcpA CodY interaction,CA-MHB;None;exp;JE2;marR_TF::bursa
CM_11,CcpA CodY interaction,CA-MHB;None;exp;JE2;ilvA::bursa
CM_12,CcpA CodY interaction,CA-MHB;None;exp;JE2;ilvA::bursa
CM_13,CcpA CodY interaction,CA-MHB;2g/Lglucose;exp;JE2;ilvA::bursa


In [44]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

True
True


In [47]:
df_metadata.to_csv(path.join(data_dir,'metadata.tsv'),sep='\t')

## Check your TRN

Each row of the TRN file represents a regulatory interaction.  
**Your TRN file must have the following columns:**
1. `regulator` - Name of regulator (`/` or `+` characters will be converted to `;`)
1. `gene_id` - Locus tag of gene being regulated

The following columns are optional, but are helpful to have:
1. `regulator_id` - Locus tag of regulator
1. `gene_name` - Name of gene (can automatically update this using `name2num`)
1. `direction` - Direction of regulation ('+' for activation, '-' for repression, '?' or NaN for unknown)
1. `evidence` - Evidence of regulation (e.g. ChIP-exo, qRT-PCR, SELEX, Motif search)
1. `PMID` - Reference for regulation

You may add any other columns that could help you. TRNs may be saved as either CSV or TSV files. See below for an example:

In [54]:
df_trn = pd.read_csv(path.join(external_data,'TRN.csv'))
df_trn.head()

,regulator,TF_id,trn_type,gene_name,N315_gene_id,gene_id
0,Zur,SA1383,TF,SA2203,SA2203,USA300HOU_RS13055
1,Zur,SA1383,TF,SA0806,SA0806,USA300HOU_RS04685
2,Zur,SA1383,TF,rpsN2,SA1171,USA300HOU_RS06795
3,Zur,SA1383,TF,rpmG2,SAS042,USA300HOU_RS06790
4,Zur,SA1383,TF,znuC,SA1385,USA300HOU_RS08305


The `regulator` and `gene_id` must be filled in for each row

In [55]:
print(df_trn.regulator.notnull().all())
print(df_trn.gene_id.notnull().all())

True
True


## Load the data
You're now ready to load your IcaData object!

In [126]:
ica_data = IcaData(M = path.join(data_dir,'M.csv'),
                   A = path.join(data_dir,'A.csv'),
                   X = path.join(data_dir,'log_tpm_norm.csv'),
                   gene_table = path.join(data_dir,'gene_info.csv'),
                   sample_table = path.join(data_dir,'metadata.tsv'),
                   trn = path.join(external_data,'TRN.csv'),
                   optimize_cutoff=True)

# Regulatory iModulons
Use `compute_trn_enrichment` to automatically check for Regulatory iModulons. The more complete your TRN, the more regulatory iModulons you'll find.

In [130]:
ica_data.compute_trn_enrichment()

,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,1,FruR,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,3.0,3.0,3.0,1.0
1,3,WalR,1.977615e-13,3.955230e-13,0.800000,0.076433,0.139535,12.0,157.0,15.0,1.0
2,5,Genomic Island 9,6.486677e-39,1.946003e-38,0.838710,0.351351,0.495238,26.0,74.0,31.0,1.0
3,5,VraR,3.907315e-06,5.860973e-06,0.193548,0.142857,0.164384,6.0,42.0,31.0,1.0
4,6,Purine,2.523321e-11,7.569963e-11,0.500000,1.000000,0.666667,4.0,4.0,8.0,1.0
5,9,TPP,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,7.0,7.0,7.0,1.0
6,10,PurR,1.365061e-29,2.730122e-29,0.857143,0.857143,0.857143,12.0,14.0,14.0,1.0
7,11,VraS2,9.312770e-09,2.793831e-08,0.500000,0.133333,0.210526,6.0,45.0,12.0,1.0
8,13,CcpA,1.213330e-81,2.426660e-80,0.654545,0.595041,0.623377,72.0,121.0,110.0,1.0
9,13,MalR,1.341246e-13,1.341246e-12,0.081818,1.000000,0.151261,9.0,9.0,110.0,1.0


You can also search for AND/OR combinations of regulators using the `max_regs` argument.

Regulator enrichments can be directly saved to the `imodulon_table` using the `save` argument. This saves the enrichment with the lowest q-value to the table.

In [132]:
# First search for regulator enrichments with 2 regulators
ica_data.compute_trn_enrichment(max_regs=2,save=True)

# Next, search for regulator enrichments with just one regulator. This will supercede the 2 regulator enrichments.
ica_data.compute_trn_enrichment(max_regs=1,save=True)

,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,1,FruR,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,3.0,3.0,3.0,1.0
1,3,WalR,1.977615e-13,3.955230e-13,0.800000,0.076433,0.139535,12.0,157.0,15.0,1.0
2,5,Genomic Island 9,6.486677e-39,1.946003e-38,0.838710,0.351351,0.495238,26.0,74.0,31.0,1.0
3,5,VraR,3.907315e-06,5.860973e-06,0.193548,0.142857,0.164384,6.0,42.0,31.0,1.0
4,6,Purine,2.523321e-11,7.569963e-11,0.500000,1.000000,0.666667,4.0,4.0,8.0,1.0
5,9,TPP,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,7.0,7.0,7.0,1.0
6,10,PurR,1.365061e-29,2.730122e-29,0.857143,0.857143,0.857143,12.0,14.0,14.0,1.0
7,11,VraS2,9.312770e-09,2.793831e-08,0.500000,0.133333,0.210526,6.0,45.0,12.0,1.0
8,13,CcpA,1.213330e-81,2.426660e-80,0.654545,0.595041,0.623377,72.0,121.0,110.0,1.0
9,13,MalR,1.341246e-13,1.341246e-12,0.081818,1.000000,0.151261,9.0,9.0,110.0,1.0


The list of regulatory iModulons are shown below

In [133]:
regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]
print(len(ica_data.imodulon_table),'Total iModulons')
print(len(regulatory_imodulons),'Regulatory iModulons')
regulatory_imodulons

76 Total iModulons
42 Regulatory iModulons


,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
1,FruR,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,3.0,3.0,3.0,1.0
3,WalR,1.977615e-13,3.955230e-13,0.800000,0.076433,0.139535,12.0,157.0,15.0,1.0
5,Genomic Island 9,6.486677e-39,1.946003e-38,0.838710,0.351351,0.495238,26.0,74.0,31.0,1.0
6,Purine,2.523321e-11,7.569963e-11,0.500000,1.000000,0.666667,4.0,4.0,8.0,1.0
9,TPP,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,7.0,7.0,7.0,1.0
10,PurR,1.365061e-29,2.730122e-29,0.857143,0.857143,0.857143,12.0,14.0,14.0,1.0
11,VraS2,9.312770e-09,2.793831e-08,0.500000,0.133333,0.210526,6.0,45.0,12.0,1.0
13,CcpA,1.213330e-81,2.426660e-80,0.654545,0.595041,0.623377,72.0,121.0,110.0,1.0
15,CggR,3.789150e-15,3.789150e-15,1.000000,0.833333,0.909091,5.0,6.0,5.0,1.0
17,CcpA,2.460824e-06,2.460824e-06,0.714286,0.041322,0.078125,5.0,121.0,7.0,1.0


You can rename iModulons in this jupyter notebook, or you can save the iModulon table as a CSV and edit it in Excel.

If two iModulons have the same regulator (e.g. 'Reg'), they will be named 'Reg-1' and 'Reg-2'

In [134]:
ica_data.rename_imodulons(regulatory_imodulons.regulator.to_dict())
ica_data.imodulon_table.head()

,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FruR,FruR,0.000000e+00,0.000000e+00,1.0,1.000000,1.000000,3.0,3.0,3.0,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WalR-1,WalR,1.977615e-13,3.955230e-13,0.8,0.076433,0.139535,12.0,157.0,15.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]

# Functional iModulons

GO annotations and KEGG pathways/modules were generated in the 1_create_the_gene_table.ipynb notebook. Enrichments will be calculated in this notebook, and further curated in the 3_manual_iModulon_curation notebook.

## GO Enrichments

First load the Gene Ontology annotations

In [136]:
DF_GO = pd.read_csv(path.join(work_dir, 'external', 'eggNOG_annotations.txt'), sep='\t', skiprows=4)
DF_GO = DF_GO.rename(columns={'#query': 'gene_id'})

In [137]:
DF_GO_enrich = ica_data.compute_annotation_enrichment(DF_GO,'COG_category')

In [138]:
DF_GO_enrich.head()

,imodulon,COG_category,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,0,E,0.000829,0.051398,0.315789,0.032967,0.059701,6.0,182.0,19.0
1,WalR-1,M,0.000045,0.002784,0.400000,0.042553,0.076923,6.0,141.0,15.0
2,Genomic Island 9-1,S,0.000213,0.013219,0.516129,0.025890,0.049307,16.0,618.0,31.0
3,Purine,F,0.000002,0.000117,0.625000,0.052632,0.097087,5.0,95.0,8.0
4,TPP,H,0.000760,0.047127,0.428571,0.036145,0.066667,3.0,83.0,7.0


## KEGG Enrichments

### Load KEGG mapping
The `kegg_mapping.csv` file contains KEGG orthologies, pathways, modules, and reactions. Only pathways and modules are relevant to iModulon characterization.

In [139]:
DF_KEGG = pd.read_csv(path.join(external_data,'kegg_mapping.csv'),index_col=0)
print(DF_KEGG.database.unique())
DF_KEGG.head()

['KEGG_ko' 'KEGG_Pathway' 'KEGG_Module' 'KEGG_Reaction']


,gene_id,database,kegg_id
10,USA300HOU_RS00050,KEGG_ko,-
11,USA300HOU_RS00055,KEGG_ko,-
13,USA300HOU_RS00065,KEGG_ko,-
14,USA300HOU_RS00070,KEGG_ko,-
20,USA300HOU_RS00110,KEGG_ko,-


In [140]:
kegg_pathways = DF_KEGG[DF_KEGG.database == 'KEGG_Pathway']
kegg_modules = DF_KEGG[DF_KEGG.database == 'KEGG_Module']

### Perform enrichment
Uses the `compute_annotation_enrichment` function

In [141]:
DF_pathway_enrich = ica_data.compute_annotation_enrichment(kegg_pathways,'kegg_id')
DF_module_enrich = ica_data.compute_annotation_enrichment(kegg_modules,'kegg_id')

In [142]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,0,map00791,2.493127e-07,0.000050,0.157895,1.000000,0.272727,3.0,3.0,19.0
1,0,map00220,3.073765e-04,0.030584,0.157895,0.142857,0.150000,3.0,21.0,19.0
2,FruR,map00051,9.958511e-05,0.019817,0.666667,0.117647,0.200000,2.0,17.0,3.0
3,4,map00260,3.619166e-04,0.072021,0.666667,0.062500,0.114286,2.0,32.0,3.0
4,Purine,map00230,1.695523e-05,0.003374,0.500000,0.060606,0.108108,4.0,66.0,8.0


### Convert KEGG IDs to human-readable names

In [143]:
for idx,key in tqdm(DF_pathway_enrich.kegg_id.items(),total=len(DF_pathway_enrich)):
    text = REST.kegg_find('pathway',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_pathway_enrich.loc[idx,'pathway_name'] = name
    except AttributeError:
        DF_pathway_enrich.loc[idx,'pathway_name'] = None
    
for idx,key in tqdm(DF_module_enrich.kegg_id.items(),total=len(DF_module_enrich)):
    text = REST.kegg_find('module',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_module_enrich.loc[idx,'module_name'] = name
    except AttributeError:
        DF_module_enrich.loc[idx,'module_name'] = None

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

In [144]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,pathway_name
0,0,map00791,2.493127e-07,0.000050,0.157895,1.000000,0.272727,3.0,3.0,19.0,Atrazine degradation
1,0,map00220,3.073765e-04,0.030584,0.157895,0.142857,0.150000,3.0,21.0,19.0,Arginine biosynthesis
2,FruR,map00051,9.958511e-05,0.019817,0.666667,0.117647,0.200000,2.0,17.0,3.0,Fructose and mannose metabolism
3,4,map00260,3.619166e-04,0.072021,0.666667,0.062500,0.114286,2.0,32.0,3.0,"Glycine, serine and threonine metabolism"
4,Purine,map00230,1.695523e-05,0.003374,0.500000,0.060606,0.108108,4.0,66.0,8.0,Purine metabolism


In [145]:
DF_module_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,module_name
0,4,M00555,7.348163e-07,1.873782e-04,0.666667,1.000000,0.800000,2.0,2.0,3.0,"Betaine biosynthesis, choline => betaine"
1,Purine,M00050,6.829507e-05,1.741524e-02,0.250000,0.400000,0.307692,2.0,5.0,8.0,"Guanine ribonucleotide biosynthesis, IMP => GD..."
2,7,M00725,8.801943e-10,2.244495e-07,0.571429,0.500000,0.533333,4.0,8.0,7.0,Cationic antimicrobial peptide (CAMP) resistan...
3,TPP,M00127,9.005102e-09,2.296301e-06,0.428571,1.000000,0.600000,3.0,3.0,7.0,"Thiamine biosynthesis, prokaryotes, AIR (+ DXP..."
4,TPP,M00582,2.548499e-06,3.249336e-04,0.428571,0.230769,0.300000,3.0,13.0,7.0,None


## Save files

In [153]:
DF_GO_enrich['source'] = 'EggNOG'
DF_pathway_enrich['source'] = 'KEGG pathways'
DF_module_enrich['source'] = 'KEGG modules'

DF_GO_enrich.rename({'COG_category':'annotation'},axis=1, inplace=True)
DF_pathway_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
DF_module_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)

DF_enrichments = pd.concat([DF_pathway_enrich, DF_module_enrich])
DF_enrichments.to_csv(path.join(data_dir,'functional_enrichments.csv'))

# Check for single gene iModulons

Some iModulons are dominated by a single, high-coefficient gene. These iModulons may result from:
1. Overdecomposition of the dataset to identify noisy genes
1. Artificial knock-out of single genes
1. Regulons with only one target gene

No matter what causes these iModulons, it is important to be aware of them. The find_single_gene_imodulons function identifies iModulons that are likely dominated by a single gene.

The iModulons identified by ``find_single_gene_imodulons`` may contain more than one gene, since a threshold-agnostic method is used to identify these iModulons.

In [154]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)
len(sg_imods)

8

In [155]:
for i,mod in enumerate(sg_imods):
    ica_data.rename_imodulons({mod:'SG_'+str(i+1)})

In [156]:
ica_data.imodulon_table[ica_data.imodulon_table.single_gene == True]

,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs,single_gene
SG_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
SG_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
SG_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
SG_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
SG_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
SG_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
SG_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
SG_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


# Save iModulon object

In [157]:
from pymodulon.util import explained_variance
from pymodulon.io import *

In [158]:
# Add iModulon sizes and explained variance
for im in ica_data.imodulon_names:
    ica_data.imodulon_table.loc[im,'imodulon_size'] = len(ica_data.view_imodulon(im))
    ica_data.imodulon_table.loc[im,'explained_variance'] = explained_variance(ica_data,imodulons=im)

This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [160]:
save_to_json(ica_data, path.join(work_dir,'interim','saureus_raw.json.gz'))

If you prefer to view and edit your iModulon table in excel, save it as a CSV and reload the iModulon as before

In [82]:
ica_data.imodulon_table.to_csv(path.join(data_dir,'imodulon_table_raw.csv'))